In [16]:
import os
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
import zipfile

In [17]:
base_dir = r'C:\Users\Weis\Desktop\Courses\ephy_data304\data'

In [18]:
zip_path = os.path.join(base_dir, 'assignment_1', 'raw', 'downloaded_class_submissions.zip')
extracted_dir = os.path.join(base_dir, 'assignment_1', 'raw', 'downloaded_class_submissions')

In [19]:
if not os.path.exists(extracted_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)

In [ ]:
nested_folder = os.path.join(extracted_dir, 'downloaded_class_submissions')

In [ ]:
html_files = []
for root, dirs, files in os.walk(nested_folder):
    for file in files:
        if file.endswith('.html'):
            html_files.append(os.path.join(root, file))

In [ ]:
db_path = os.path.join(base_dir, 'assignment_3', 'altered', 'parsed_bios.sqlite')

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS names (name_id INTEGER PRIMARY KEY, name TEXT UNIQUE)''')
cursor.execute('''CREATE TABLE IF NOT EXISTS favorites (favorite_id INTEGER PRIMARY KEY, name_id INTEGER, category TEXT, favorite TEXT, 
                  FOREIGN KEY(name_id) REFERENCES names(name_id), UNIQUE(name_id, category, favorite))''')

# conn.commit()

In [9]:
cursor.execute("DELETE FROM favorites")
conn.commit()

In [15]:
for file_name in html_files:
    html_path = os.path.join(extracted_dir, file_name)

    # Read HTML
    with open(html_path, 'r', encoding='utf-8') as html_file:
        html_content = html_file.read()

    # Parse HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract name from <h1>
    h1_tag = soup.find('h1')
    if h1_tag:
        name_value = h1_tag.text.strip()
    else:
        continue  # Skip file if no name found

    # Insert name into 'names' table if not already present
    cursor.execute("INSERT OR IGNORE INTO names (name) VALUES (?)", (name_value,))

    # Retrieve name_id for the current name
    cursor.execute("SELECT name_id FROM names WHERE name = ?", (name_value,))
    name_id = cursor.fetchone()[0]

    # Extract table
    table = soup.find('table')
    if not table:
        continue

    # Extract data from table
    for row in table.find_all('tr')[1:]:  # Skip header row
        cols = [td.text.strip() for td in row.find_all('td')]
        if len(cols) == 2:  # Ensure correct data format
            category, favorite = cols

            # Check if this combination already exists in the favorites table
            cursor.execute('''SELECT COUNT(*) FROM favorites 
                              WHERE name_id = ? AND category = ? AND favorite = ?''', 
                           (name_id, category, favorite))
            count = cursor.fetchone()[0]

            # If the combination doesn't exist, insert it
            if count == 0:
                cursor.execute('''INSERT INTO favorites (name_id, category, favorite)
                                  VALUES (?, ?, ?)''', (name_id, category, favorite))

    # Commit all changes after processing the file
    conn.commit()

# Final commit after all files are processed (if necessary)
conn.commit()

cursor.execute("SELECT * FROM names")
names = cursor.fetchall()
print(f"\nNames Table: {names}")

cursor.execute("SELECT * FROM favorites")
favorites = cursor.fetchall()
print(f"\nFavorites Table: {favorites}")

conn.close()

ProgrammingError: Cannot operate on a closed database.